## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [71]:
import random
import numpy as np
import pandas as pd
import line_profiler

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [53]:
A=np.random.randint(0, 1001, size = 1000000)
B = A+100
print(A)
print(B)
B.mean()

[574 853 512 ... 658 443 652]
[674 953 612 ... 758 543 752]


600.006792

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [54]:
import string
df = pd.DataFrame(np.random.randint(-100,1000,size=(2000000, 4)), columns=list('1234'))
lst_letter = [random.choice(string.ascii_letters) for _ in range(2000000)]
df['key'] = lst_letter
df.head()
df[(df['key'] == 'a') | (df['key'] == 'b') | (df['key'] == 'c') | (df['key'] == 'd') | (df['key'] == 'e') | (df['key'] == 'A') | (df['key'] == 'B') | (df['key'] == 'C') | (df['key'] == 'D') | (df['key'] == 'E')]


,1,2,3,4,key
4,962,469,38,457,A
8,423,158,260,668,B
14,-100,755,637,900,b
16,691,-20,952,552,B
17,971,927,536,718,C
...,...,...,...,...,...
1999963,938,463,945,650,E
1999968,109,-92,280,262,b
1999970,137,568,411,130,C
1999981,608,-22,525,5,D


## Лабораторная работа 3

In [69]:
!pip install line_profiler

In [79]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [56]:
import pandas as pd

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [57]:
recipes = pd.read_csv("recipes_sample.csv", parse_dates=["submitted"])
reviews = pd.read_csv("reviews_sample.csv", index_col=0, parse_dates=["date"])

In [58]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [59]:
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [64]:
%%time
def mean_rating_A(): #A. С использованием метода DataFrame.iterrows исходной таблицы;
    s = 0
    k = 0
    for _, row in reviews.iterrows():
        if row['date'].year == 2010:
            s += row['rating']
            k += 1
    return s/k

print('Среднее значение столбца rating из таблицы reviews',mean_rating_A())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
CPU times: total: 3.55 s
Wall time: 3.55 s


In [65]:
%%time
def mean_rating_B(): #Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;
    s = 0
    k = 0
    dfB = reviews[reviews['date'].dt.year == 2010]
    for _, row in dfB.iterrows():
        s += row['rating']
    k = dfB.shape[0]
    return s/k
print('Среднее значение столбца rating из таблицы reviews',mean_rating_B())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
CPU times: total: 312 ms
Wall time: 311 ms


In [68]:
%%time
def mean_rating_C(): # С использованием метода Series.mean.
    return reviews[reviews['date'].dt.year == 2010].rating.mean()
print('Среднее значение столбца rating из таблицы reviews',mean_rating_C())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
CPU times: total: 15.6 ms
Wall time: 14.9 ms


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [81]:
%lprun -f mean_rating_A mean_rating_A() #для буквы А

#### A. С использованием метода DataFrame.iterrows исходной таблицы

Timer unit: 1e-07 s

Total time: 11.6087 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1          8.0      8.0      0.0  
     3         1          2.0      2.0      0.0  
     4    126696   97012265.0    765.7     83.6  
     5    114602   17522373.0    152.9     15.1  
     6     12094    1511529.0    125.0      1.3  
     7     12094      41302.0      3.4      0.0  
     8         1          5.0      5.0      0.0

In [82]:
%lprun -f mean_rating_B mean_rating_B() #для буквы Б

####  Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год

Timer unit: 1e-07 s

Total time: 1.0978 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1          4.0      4.0      0.0  
     3         1          2.0      2.0      0.0  
     4         1     172069.0 172069.0      1.6  
     5     12094    8992641.0    743.6     81.9  
     6     12094    1813247.0    149.9     16.5  
     7         1         58.0     58.0      0.0  
     8         1          7.0      7.0      0.0

In [83]:
%lprun -f mean_rating_C mean_rating_C() #для буквы B

#### В. С использованием метода Series.mean

Timer unit: 1e-07 s

Total time: 0.0159564 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1     159564.0 159564.0    100.0

##### Мы видим,что быстрее всего работает вариант B, который реализован без циклов.

##### Рассмотрим варинат Б без цикла

In [86]:
%%time
def mean_rating_B2(): #Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;
    s = 0
    k = 0
    dfB = reviews[reviews['date'].dt.year == 2010]
    return sum(dfB.rating)/dfB.shape[0]
print('Среднее значение столбца rating из таблицы reviews',mean_rating_B2())

Среднее значение столбца rating из таблицы reviews 4.4544402182900615
CPU times: total: 31.2 ms
Wall time: 12.3 ms


In [88]:
%lprun -f mean_rating_B2 mean_rating_B2() #для буквы Б, реализованной без цикла

Timer unit: 1e-07 s

Total time: 0.0141376 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1          6.0      6.0      0.0  
     3         1          2.0      2.0      0.0  
     4         1     135216.0 135216.0     95.6  
     5         1       6152.0   6152.0      4.4

###### Мы смогли ускорить работу функции Б: было - 1.0978 s, а стало - 0.0141376 s

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [101]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [93]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 37.3144 s
File: C:\Users\Admin\AppData\Local\Temp\ipykernel_7824\2826575548.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1          5.0      5.0      0.0      word_reviews = {}
     3    126679  101181044.0    798.7     27.1      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   34694023.0    273.9      9.3          recipe_id, review = row['recipe_id'], row['review']
     5    126679    3677796.0     29.0      1.0          words = review.split(' ')
     6   6792010   15218272.0      2.2      4.1          for word in words:
     7   6617066   19798329.0      3.0      5.3              if word not in word_reviews:
     8    174944     658136.0      3.8      0.2                  word_reviews[word] = []
     9   6792010   26262381.0      3.9      7.0              word_reviews[word].append(recipe_id)
    10                                               
    11         1          3.0      3.0      0.0      word_reviews_count = {}
    12    126679   98811199.0    780.0     26.5      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679   19237329.0    151.9      5.2          review = row['review']
    14    126679    3623239.0     28.6      1.0          words = review.split(' ')
    15   6792010   15917286.0      2.3      4.3          for word in words:
    16   6792010   34065144.0      5.0      9.1              word_reviews_count[word] = len(word_reviews[word])
    17         1          3.0      3.0      0.0      return word_reviews_count

Видно, что много времени занимают циклы, поэтому я совместила два цикла в один и использую только один словарь

In [107]:
def get_word_reviews_count2(df):
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews_count:
                word_reviews_count[word] = 0
            word_reviews_count[word] += 1
    return word_reviews_count

In [109]:
%lprun -f get_word_reviews_count2 get_word_reviews_count2(reviews)

Timer unit: 1e-07 s

Total time: 18.9685 s
File: C:\Users\Admin\AppData\Local\Temp\ipykernel_7824\1845820076.py
Function: get_word_reviews_count2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count2(df):
     2         1          7.0      7.0      0.0      word_reviews_count = {}
     3    126679  103419117.0    816.4     54.5      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   20206979.0    159.5     10.7          review = row['review']
     5    126679    3870092.0     30.6      2.0          words = review.split(' ')
     6   6792010   15729259.0      2.3      8.3          for word in words:
     7   6617066   20351648.0      3.1     10.7              if word not in word_reviews_count:
     8    174944     599653.0      3.4      0.3                  word_reviews_count[word] = 0
     9   6792010   25508624.0      3.8     13.4              word_reviews_count[word] += 1
    10         1          3.0      3.0      0.0      return word_reviews_count

##### Итог: время сократилось(было 37.3144 s, стало 18.9685 s )

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [142]:
reviews = reviews[reviews.rating !=0]
reviews['mean'] = reviews.groupby('recipe_id').rating.transform('mean') #transform вычисление среднего рейтинга для каждой группы
reviews.head()

,user_id,recipe_id,date,rating,review,mean
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...,4.818182
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...,5.000000
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy...",4.000000
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...,4.555556
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...,5.000000


In [110]:
import numba

In [133]:
%%time
def mape_A(df):
    s = 0
    sum_rows = 0
    for index, row in df.iterrows():
        s += abs(row["rating"] - row['mean']) / row['rating']
        sum_rows += 1
    return s/sum_rows
mape_A(reviews)


CPU times: total: 3.92 s
Wall time: 3.9 s


0.11171550259058839

In [159]:
%%time
rating = reviews['rating'].to_list()
mean = reviews['mean'].to_list() 
@numba.njit(nopython=True)
def mape_B(rating, mean): 
    s = 0
    sum_rows = 0
    for i in range(len(rating)):
        s += abs(rating[i] - mean[i]) / rating[i]
        sum_rows += 1
    return s/sum_rows
mape_B(rating, mean)

CPU times: total: 375 ms
Wall time: 377 ms


0.11171550259058839

In [161]:
%%time
def mape_C(df):
    rating = df['rating'].to_numpy()
    mean = df['mean'].to_numpy() 
    mape = abs(rating - mean) / rating
    return mape.mean()
mape_C(reviews)

CPU times: total: 0 ns
Wall time: 998 µs


0.11171550259058086

In [163]:
%%time
rating = reviews['rating'].to_numpy()
mean = reviews['mean'].to_numpy() 

@numba.jit
def mape_D(rating, mean):
    mape = abs(rating - mean) / rating
    return mape.mean()
mape_D(rating, mean)

CPU times: total: 46.9 ms
Wall time: 49.9 ms


0.11171550259058086

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4